#### Latent Semantic Indexing Recommender System
In order to improve on the TFIDF model, this version transforms the vectors using LSI. This allows us to capture the semantic meaning between words like "musical instrument" rather than ranking them as two separate words like "music" and "instrument". This plays a significant role in increasing precision, as it removes scenarios where a class like "334516 - Analytical Laboratory Instrument Manufacturing" will be weight higher due to the presence of the word "instrument"

In [293]:
import pandas as pd
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re
from sklearn.decomposition import TruncatedSVD

In [294]:
stop_words = set(stopwords.words("english"))

PATTERN_S = re.compile("\'s")  # matches `'s` from text  
PATTERN_RN = re.compile("\\r\\n\\b") #matches `\r` and `\n`
PATTERN_PUNC = re.compile(r"[^\w\s]") # matches all non 0-9 A-z whitespace 


def clean_text(text):
    """
    Series of cleaning. String to lower case, remove non words characters and numbers (punctuation, curly brackets etc).
        text (str): input text
    return (str): modified initial text
    """
    text = text.lower()  # lowercase text
    # replace the matched string with ' '
    text = re.sub(PATTERN_S, ' ', text)
    text = re.sub(PATTERN_RN, ' ', text)
    text = re.sub(PATTERN_PUNC, ' ', text)
    return text

def tokenizer(description, stop_words, normalization):
    
    if normalization == 'lemmatize':
        # tokenize and lemmatize text
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(w) for w in word_tokenize(description)]
        
    elif normalization == 'stem':
        # tokenize and stem text
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(w) for w in word_tokenize(description)]
    
   # remove tokens length of 2 or below and make all lowercase and remove stop words
    tokens = [w.lower() for w in tokens if (w.lower() not in stop_words) and (len(w) > 2) and(w.isalpha())]
    
    return tokens    
    
def process_query(query, normalization):
    stop_words = set(stopwords.words("english"))
    
    return tokenizer(clean_text(query), stop_words, normalization)

In [295]:
# code example taken from https://towardsdatascience.com/build-a-text-recommendation-system-with-python-e8b95d9f251c
def retrieve_top_n(m, max_docs):
    # return the sum on all tokens of cosinus for each sentence
    if len(m.shape) > 1:
        cos_sim = np.mean(m, axis=0) 
    else: 
        cos_sim = m
    index = np.argsort(cos_sim)[::-1] # from highest idx to smallest score \
    mask = np.ones(len(cos_sim))
    mask = np.logical_or(cos_sim[index] != 0, mask) #eliminate 0 cosine distance
    best_index = index[mask][:max_docs]  
    return best_index

In [296]:
class LsiTfidfRecommenderSystem:
    def __init__(self, docs, num_concepts=100, alpha=1.0, beta=0.75, gamma=0.15):
        self.alpha, self.beta, self.gamma = alpha, beta, gamma
        
        # create a doc-term matrix out of our doc collection
        self.vec = TfidfVectorizer()
        doc_term_mat = self.vec.fit_transform([" ".join(docs[doc_id]) for doc_id in docs])
        self.svd = TruncatedSVD(n_components=num_concepts, random_state=42)
        result = self.svd.fit_transform(doc_term_mat)
        
        self.q_vecs = {}
        
        self.doc_vecs = result # document vectors in a matrix
        
    def retrieve_docs(self, query, max_docs=10, normalization='lemmatize'):
        query = ' '.join(process_query(query, normalization))
        
        if query not in self.q_vecs:
            q_vec = self.vec.transform([query])
            lsi_transform = self.svd.transform(q_vec)
            self.q_vecs[query] = lsi_transform
        
        ret_docs = {}
        
        mat = cosine_similarity(self.q_vecs[query], self.doc_vecs)
        best_index = retrieve_top_n(mat, max_docs=max_docs)
        
        return best_index

In [297]:
df = pd.read_pickle(r'assets/processed_df.pkl')
docs = dict(zip(df['naics'], df['lemmatized']))

In [298]:
lsi_model = LsiTfidfRecommenderSystem(docs)

In [299]:
best_index = lsi_model.retrieve_docs('lawyer office', normalization='lemmatize')

In [300]:
naics_titles = pd.read_excel('assets/6-digit_2017_Codes.xlsx')
naics_titles['naics'] = naics_titles['naics'].astype(str)

In [301]:
df = df.merge(naics_titles, on='naics', how='outer')

In [302]:
df.iloc[best_index][['naics', 'title']]

,naics,title
908,621399,Offices of All Other Miscellaneous Health Prac...
907,621391,Offices of Podiatrists
906,621340,"Offices of Physical, Occupational and Speech T..."
789,541191,Title Abstract and Settlement Offices
790,541199,All Other Legal Services
787,541110,Offices of Lawyers
920,621999,All Other Miscellaneous Ambulatory Health Care...
904,621320,Offices of Optometrists
900,621111,Offices of Physicians (except Mental Health Sp...
901,621112,"Offices of Physicians, Mental Health Specialists"


In [304]:
best_index = lsi_model.retrieve_docs('lawyer office', normalization='stem')
df.iloc[best_index][['naics', 'title']]

,naics,title
789,541191,Title Abstract and Settlement Offices
790,541199,All Other Legal Services
787,541110,Offices of Lawyers
788,541120,Offices of Notaries
908,621399,Offices of All Other Miscellaneous Health Prac...
794,541219,Other Accounting Services
791,541211,Offices of Certified Public Accountants
792,541213,Tax Preparation Services
793,541214,Payroll Services
802,541380,Testing Laboratories


In [243]:
relevant_naics = pd.read_pickle('assets/relevant_naics_df.pkl')

In [244]:
relevant_naics.head()

,query,relevant_naics
0,Home improvement store,"[444110, 444120, 444130, 444190]"
1,Diesel fuel supplier,"[424710, 424720]"
2,Church,[813110]
3,Farm,"[115116, 115111, 115112, 115113, 115114, 115115]"
4,Seed supplier,"[424910, 424920, 424930, 424940, 424950, 424990]"


In [245]:
max_docs = 10

In [246]:
def calc_pre_rec_at_n(ret_docs, reljudges, n=-1):
    """
    Calculate precision and recall at n for each query in ret_docs
    """
    
    pre_at_n, rec_at_n = {}, {}
    
    for k, v in ret_docs.items():
        if n > -1 and n <= len(ret_docs):
            s1 = set(v[:n])
        else:
            s1 = set(v)
        s2 = reljudges[k]
        precision = len(s1.intersection(s2)) / len(s1)
        recall = len(s1.intersection(s2)) / len(reljudges[k])
        pre_at_n[k] = round(precision, 3)
        rec_at_n[k] = round(recall, 3)
    return pre_at_n, rec_at_n

In [247]:
def calc_avg_pre(ret_docs, reljudges, cutoff=-1):
    """
    Calculate (mean) average precision for each query in ret_docs
    """
    
    avg_pre, mean_avg_pre = {}, None
    for k, v in ret_docs.items():
        total_rel = 0
        total = 0
        avg_prec = 0
        for i, doc in enumerate(v):
            if doc in reljudges[k] and cutoff == -1:
                total_rel += 1
                total += 1
                precision = total_rel/total
            elif doc in reljudges[k] and i+1 <= cutoff:
                total_rel += 1
                total += 1
                precision = total_rel/total
            else:
                total += 1
                precision = 0
            avg_prec += precision

        avg_pre[k] = round(avg_prec/len(reljudges[k]), 3,)
    
    mean_avg_pre = round(sum(avg_pre.values()) / len(avg_pre), 3)
        
    return avg_pre, mean_avg_pre

In [248]:
import math

def calc_NDCG_at_n(ret_docs, reljudges, n=-1, base=2):
    """
    Calculate NDCG at n for each query in ret_docs
    """
    
    ndcg = {}
    
    for k, v in ret_docs.items():
        
        counts = list(reversed([x for x in range(2,len(reljudges[k])+2)]))
        ideals = {reljudges[k][i]: counts[i] for i in range(len(reljudges[k]))}
        
        add_ons = {}
        if len(v) > len(reljudges[k]):
            for i in range(len(v)-len(reljudges[k])):
                add_ons[i] = 1
        ideals.update(add_ons)
        nums = list(map(ideals.get, v))
        
        systems = {}
        for i, doc in enumerate(v):
            if nums[i] == None:
                systems[doc] = 1
            else:
                systems[doc] = nums[i]
                
        ideal_order = {}
        if n != -1:
            for i, (key, value) in enumerate(ideals.items()):
                if i < n:
                    ideal_order[key] = value
        else:
            ideal_order = ideals
        
        add_ons = {}
        
        
        log = 0
        for i, (doc, rank) in enumerate(ideal_order.items()):
            if i >= len(v):
                break
            elif i < base:
                log += rank
            else:
                log += rank/math.log(i+1, base)

                
        system_order = {}
        if n != -1:
            for i, (key, value) in enumerate(systems.items()):
                if i < n:
                    system_order[key] = value
        else:
            system_order = systems
        
        
        
        system_log = 0
        for i, (doc, rank) in enumerate(system_order.items()):
            if i >= len(v):
                break
            elif i < base:
                system_log += rank
            else:
                system_log += rank/math.log(i+1, base)
        system_log / log
        ndcg[k] = round(system_log / log, 3)
        
    
    return ndcg

In [249]:
ret_docs_dic = {}
queries_dic = {}

for query in relevant_naics.iterrows():
    query_name =  ' '.join(process_query(query[1]['query'], 'lemmatize'))
    ret_docs_index = lsi_model.retrieve_docs(query_name, max_docs=max_docs, normalization='lemmatize')
    ret_docs = df.iloc[ret_docs_index]['naics'].tolist()

    query_docs = query[1]['relevant_naics']
    ret_docs_dic[query_name] = ret_docs
    queries_dic[query_name] = query_docs

In [250]:
ret_docs_dic

{'home improvement store': ['442299',
  '442291',
  '442210',
  '453998',
  '453991',
  '453930',
  '453910',
  '453920',
  '453310',
  '453220'],
 'diesel fuel supplier': ['447190',
  '447110',
  '336310',
  '445120',
  '454310',
  '454390',
  '441310',
  '221112',
  '454210',
  '424720'],
 'church': ['337127',
  '337121',
  '337124',
  '337122',
  '337125',
  '337214',
  '337110',
  '337211',
  '337910',
  '337920'],
 'farm': ['115115',
  '115210',
  '115310',
  '115113',
  '115116',
  '115111',
  '115112',
  '115114',
  '113110',
  '114210'],
 'seed supplier': ['111191',
  '111199',
  '111998',
  '111992',
  '111991',
  '111219',
  '111120',
  '111419',
  '111160',
  '111150'],
 'accountant': ['541211',
  '541219',
  '541213',
  '541214',
  '541110',
  '541199',
  '541120',
  '541191',
  '541490',
  '541410'],
 'trucking company': ['484121',
  '484230',
  '484122',
  '484220',
  '484110',
  '484210',
  '551114',
  '551111',
  '551112',
  '561110'],
 'exporter': ['522298',
  '522292'

In [251]:
queries_dic

{'home improvement store': ['444110', '444120', '444130', '444190'],
 'diesel fuel supplier': ['424710', '424720'],
 'church': ['813110'],
 'farm': ['115116', '115111', '115112', '115113', '115114', '115115'],
 'seed supplier': ['424910', '424920', '424930', '424940', '424950', '424990'],
 'accountant': ['541219', '541211', '541213', '541214'],
 'trucking company': ['484110', '484121', '484122'],
 'exporter': ['522293',
  '522291',
  '522292',
  '522294',
  '522298',
  '522210',
  '522220'],
 'grain elevator': ['493130', '493110', '493120', '493190'],
 'popcorn store': ['445299', '445291', '445292', '445210', '445220', '445230'],
 'agricultural service': ['115116',
  '115111',
  '115112',
  '115113',
  '115114',
  '115115'],
 'warehouse': ['493110', '493120', '493130', '493190'],
 'agricultural production': ['111998',
  '111991',
  '111992',
  '111910',
  '111920',
  '111930',
  '111940'],
 'ranch': ['112130', '112111', '112112', '112120'],
 'holding company': ['551112', '551111', '551

In [252]:
calc_pre_rec_at_n(ret_docs_dic, queries_dic, n=-1)[0]

{'home improvement store': 0.0,
 'diesel fuel supplier': 0.1,
 'church': 0.0,
 'farm': 0.6,
 'seed supplier': 0.0,
 'accountant': 0.4,
 'trucking company': 0.3,
 'exporter': 0.7,
 'grain elevator': 0.0,
 'popcorn store': 0.3,
 'agricultural service': 0.0,
 'warehouse': 0.4,
 'agricultural production': 0.1,
 'ranch': 0.0,
 'holding company': 0.3,
 'farm equipment supplier': 0.0,
 'store': 0.0,
 'grocery store': 0.0,
 'rice mill': 0.7,
 'food product supplier': 0.0,
 'accounting firm': 0.4,
 'produce market': 0.0,
 'pet supply store': 0.5,
 'wholesaler': 0.0,
 'produce wholesaler': 0.0,
 'distribution service': 0.0,
 'crop grower': 0.5,
 'addiction treatment center': 0.0,
 'natural good store': 0.0,
 'orchard': 0.2,
 'lumber store': 0.0,
 'mine': 0.0,
 'transportation service': 0.0,
 'investment company': 0.1,
 'fruit wholesaler': 0.3,
 'real estate agency': 0.1,
 'event venue': 0.0,
 'frozen dessert supplier': 0.5,
 'wine wholesaler importer': 0.2,
 'winery': 0.6,
 'aerospace company': 

In [253]:
calc_pre_rec_at_n(ret_docs_dic, queries_dic, n=-1)[1]

{'home improvement store': 0.0,
 'diesel fuel supplier': 0.5,
 'church': 0.0,
 'farm': 1.0,
 'seed supplier': 0.0,
 'accountant': 1.0,
 'trucking company': 1.0,
 'exporter': 1.0,
 'grain elevator': 0.0,
 'popcorn store': 0.5,
 'agricultural service': 0.0,
 'warehouse': 1.0,
 'agricultural production': 0.143,
 'ranch': 0.0,
 'holding company': 1.0,
 'farm equipment supplier': 0.0,
 'store': 0.0,
 'grocery store': 0.0,
 'rice mill': 1.0,
 'food product supplier': 0.0,
 'accounting firm': 1.0,
 'produce market': 0.0,
 'pet supply store': 1.0,
 'wholesaler': 0.0,
 'produce wholesaler': 0.0,
 'distribution service': 0.0,
 'crop grower': 0.714,
 'addiction treatment center': 0.0,
 'natural good store': 0.0,
 'orchard': 0.222,
 'lumber store': 0.0,
 'mine': 0.0,
 'transportation service': 0.0,
 'investment company': 0.25,
 'fruit wholesaler': 0.333,
 'real estate agency': 1.0,
 'event venue': 0.0,
 'frozen dessert supplier': 1.0,
 'wine wholesaler importer': 1.0,
 'winery': 1.0,
 'aerospace c

In [254]:
calc_avg_pre(ret_docs_dic, queries_dic, cutoff=-1)[0]

{'home improvement store': 0.0,
 'diesel fuel supplier': 0.05,
 'church': 0.0,
 'farm': 0.705,
 'seed supplier': 0.0,
 'accountant': 1.0,
 'trucking company': 0.756,
 'exporter': 0.982,
 'grain elevator': 0.0,
 'popcorn store': 0.12,
 'agricultural service': 0.0,
 'warehouse': 0.771,
 'agricultural production': 0.014,
 'ranch': 0.0,
 'holding company': 1.0,
 'farm equipment supplier': 0.0,
 'store': 0.0,
 'grocery store': 0.0,
 'rice mill': 1.0,
 'food product supplier': 0.0,
 'accounting firm': 1.0,
 'produce market': 0.0,
 'pet supply store': 0.876,
 'wholesaler': 0.0,
 'produce wholesaler': 0.0,
 'distribution service': 0.0,
 'crop grower': 0.59,
 'addiction treatment center': 0.0,
 'natural good store': 0.0,
 'orchard': 0.222,
 'lumber store': 0.0,
 'mine': 0.0,
 'transportation service': 0.0,
 'investment company': 0.042,
 'fruit wholesaler': 0.289,
 'real estate agency': 0.2,
 'event venue': 0.0,
 'frozen dessert supplier': 0.563,
 'wine wholesaler importer': 0.833,
 'winery': 1.

In [255]:
# lmmatizing performs a bit better in the concept space, perhaps because full words group into concepts al ittle differently?

In [256]:
calc_avg_pre(ret_docs_dic, queries_dic, cutoff=-1)[1]

0.28

In [257]:
lsi_maps = []
for i in range(1, 11):
    lsi_maps.append(calc_avg_pre(ret_docs_dic, queries_dic, cutoff=i)[1])

In [258]:
lsi_maps

[0.072, 0.122, 0.166, 0.202, 0.226, 0.243, 0.255, 0.266, 0.273, 0.28]

In [259]:
calc_NDCG_at_n(ret_docs_dic, queries_dic, n=-1, base=2)

{'home improvement store': 0.375,
 'diesel fuel supplier': 0.673,
 'church': 0.84,
 'farm': 0.667,
 'seed supplier': 0.244,
 'accountant': 1.0,
 'trucking company': 0.843,
 'exporter': 0.905,
 'grain elevator': 0.375,
 'popcorn store': 0.493,
 'agricultural service': 0.244,
 'warehouse': 0.859,
 'agricultural production': 0.227,
 'ranch': 0.375,
 'holding company': 1.0,
 'farm equipment supplier': 0.244,
 'store': 0.637,
 'grocery store': 0.637,
 'rice mill': 1.0,
 'food product supplier': 0.148,
 'accounting firm': 1.0,
 'produce market': 0.244,
 'pet supply store': 0.798,
 'wholesaler': 0.637,
 'produce wholesaler': 0.148,
 'distribution service': 0.203,
 'crop grower': 0.84,
 'addiction treatment center': 0.637,
 'natural good store': 0.299,
 'orchard': 0.233,
 'lumber store': 0.375,
 'mine': 0.244,
 'transportation service': 0.637,
 'investment company': 0.485,
 'fruit wholesaler': 0.447,
 'real estate agency': 0.909,
 'event venue': 0.483,
 'frozen dessert supplier': 0.728,
 'wine

# Stemming

In [260]:
docs = dict(zip(df['naics'], df['stemmed']))

In [261]:
lsi_model = LsiTfidfRecommenderSystem(docs)

In [262]:
ret_docs_dic = {}
queries_dic = {}

for query in relevant_naics.iterrows():
    query_name =  ' '.join(process_query(query[1]['query'], 'stem'))
    ret_docs_index = lsi_model.retrieve_docs(query_name, max_docs=max_docs, normalization='stem')
    ret_docs = df.iloc[ret_docs_index]['naics'].tolist()

    query_docs = query[1]['relevant_naics']
    ret_docs_dic[query_name] = ret_docs
    queries_dic[query_name] = query_docs

In [263]:
calc_pre_rec_at_n(ret_docs_dic, queries_dic, n=-1)[0]

{'home improv store': 0.0,
 'diesel fuel supplier': 0.0,
 'church': 0.0,
 'farm': 0.0,
 'seed supplier': 0.0,
 'account': 0.3,
 'truck compani': 0.3,
 'export': 0.0,
 'grain elev': 0.0,
 'popcorn store': 0.3,
 'agricultur servic': 0.0,
 'warehous': 0.4,
 'agricultur product': 0.0,
 'ranch': 0.2,
 'hold compani': 0.3,
 'farm equip supplier': 0.0,
 'store': 0.0,
 'groceri store': 0.0,
 'rice mill': 0.7,
 'food product supplier': 0.0,
 'account firm': 0.0,
 'produc market': 0.0,
 'pet suppli store': 0.5,
 'wholesal': 0.0,
 'produc wholesal': 0.0,
 'distribut servic': 0.0,
 'crop grower': 0.7,
 'addict treatment center': 0.0,
 'natur good store': 0.0,
 'orchard': 0.3,
 'lumber store': 0.0,
 'mine': 0.4,
 'transport servic': 0.0,
 'invest compani': 0.2,
 'fruit wholesal': 0.1,
 'real estat agenc': 0.1,
 'event venu': 0.0,
 'frozen dessert supplier': 0.5,
 'wine wholesal import': 0.2,
 'wineri': 0.6,
 'aerospac compani': 0.6,
 'cold storag facil': 0.4,
 'employ agenc': 0.0,
 'plant nurseri':

In [264]:
calc_pre_rec_at_n(ret_docs_dic, queries_dic, n=-1)[1]

{'home improv store': 0.0,
 'diesel fuel supplier': 0.0,
 'church': 0.0,
 'farm': 0.0,
 'seed supplier': 0.0,
 'account': 0.75,
 'truck compani': 1.0,
 'export': 0.0,
 'grain elev': 0.0,
 'popcorn store': 0.5,
 'agricultur servic': 0.0,
 'warehous': 1.0,
 'agricultur product': 0.0,
 'ranch': 0.5,
 'hold compani': 1.0,
 'farm equip supplier': 0.0,
 'store': 0.0,
 'groceri store': 0.0,
 'rice mill': 1.0,
 'food product supplier': 0.0,
 'account firm': 0.0,
 'produc market': 0.0,
 'pet suppli store': 1.0,
 'wholesal': 0.0,
 'produc wholesal': 0.0,
 'distribut servic': 0.0,
 'crop grower': 1.0,
 'addict treatment center': 0.0,
 'natur good store': 0.0,
 'orchard': 0.333,
 'lumber store': 0.0,
 'mine': 0.667,
 'transport servic': 0.0,
 'invest compani': 0.5,
 'fruit wholesal': 0.111,
 'real estat agenc': 1.0,
 'event venu': 0.0,
 'frozen dessert supplier': 1.0,
 'wine wholesal import': 1.0,
 'wineri': 1.0,
 'aerospac compani': 1.0,
 'cold storag facil': 1.0,
 'employ agenc': 0.0,
 'plant nu

In [265]:
calc_avg_pre(ret_docs_dic, queries_dic, cutoff=-1)[0]

{'home improv store': 0.0,
 'diesel fuel supplier': 0.0,
 'church': 0.0,
 'farm': 0.0,
 'seed supplier': 0.0,
 'account': 0.688,
 'truck compani': 0.589,
 'export': 0.0,
 'grain elev': 0.0,
 'popcorn store': 0.115,
 'agricultur servic': 0.0,
 'warehous': 1.0,
 'agricultur product': 0.0,
 'ranch': 0.098,
 'hold compani': 1.0,
 'farm equip supplier': 0.0,
 'store': 0.0,
 'groceri store': 0.0,
 'rice mill': 1.0,
 'food product supplier': 0.0,
 'account firm': 0.0,
 'produc market': 0.0,
 'pet suppli store': 0.876,
 'wholesal': 0.0,
 'produc wholesal': 0.0,
 'distribut servic': 0.0,
 'crop grower': 0.909,
 'addict treatment center': 0.0,
 'natur good store': 0.0,
 'orchard': 0.256,
 'lumber store': 0.0,
 'mine': 0.414,
 'transport servic': 0.0,
 'invest compani': 0.086,
 'fruit wholesal': 0.014,
 'real estat agenc': 0.2,
 'event venu': 0.0,
 'frozen dessert supplier': 0.486,
 'wine wholesal import': 0.7,
 'wineri': 1.0,
 'aerospac compani': 0.502,
 'cold storag facil': 1.0,
 'employ agenc'

In [266]:
calc_avg_pre(ret_docs_dic, queries_dic, cutoff=-1)[1]

0.282

In [267]:
calc_NDCG_at_n(ret_docs_dic, queries_dic, n=-1, base=2)

{'home improv store': 0.375,
 'diesel fuel supplier': 0.637,
 'church': 0.84,
 'farm': 0.244,
 'seed supplier': 0.244,
 'account': 0.946,
 'truck compani': 0.843,
 'export': 0.203,
 'grain elev': 0.375,
 'popcorn store': 0.481,
 'agricultur servic': 0.244,
 'warehous': 0.867,
 'agricultur product': 0.203,
 'ranch': 0.546,
 'hold compani': 1.0,
 'farm equip supplier': 0.244,
 'store': 0.637,
 'groceri store': 0.637,
 'rice mill': 1.0,
 'food product supplier': 0.148,
 'account firm': 0.375,
 'produc market': 0.244,
 'pet suppli store': 0.882,
 'wholesal': 0.637,
 'produc wholesal': 0.148,
 'distribut servic': 0.203,
 'crop grower': 0.957,
 'addict treatment center': 0.637,
 'natur good store': 0.299,
 'orchard': 0.284,
 'lumber store': 0.375,
 'mine': 0.728,
 'transport servic': 0.637,
 'invest compani': 0.498,
 'fruit wholesal': 0.214,
 'real estat agenc': 0.909,
 'event venu': 0.483,
 'frozen dessert supplier': 0.699,
 'wine wholesal import': 0.931,
 'wineri': 0.871,
 'aerospac compan

In [268]:
df = pd.read_pickle(r'assets/processed_df.pkl')
docs = dict(zip(df['naics'], df['lemmatized']))

In [274]:
concepts = [5, 10, 20, 50, 100, 150, 200, 250, 300, 350, 400]

#### Test the ideal number of concepts with the highest mAP@N score

In [275]:
for concept in concepts:
    lsi_model = LsiTfidfRecommenderSystem(docs, num_concepts=concept)
    ret_docs_dic = {}
    queries_dic = {}

    for query in relevant_naics.iterrows():
        query_name =  ' '.join(process_query(query[1]['query'], 'lemmatize'))
        ret_docs_index = lsi_model.retrieve_docs(query_name, max_docs=max_docs)
        ret_docs = df.iloc[ret_docs_index]['naics'].tolist()

        query_docs = query[1]['relevant_naics']
        ret_docs_dic[query_name] = ret_docs
        queries_dic[query_name] = query_docs
    print(f'Number of concepts: {concept} ' + str(calc_avg_pre(ret_docs_dic, queries_dic, cutoff=-1)[1]))

Number of concepts: 5 0.039
Number of concepts: 10 0.071
Number of concepts: 20 0.126
Number of concepts: 50 0.206
Number of concepts: 100 0.28
Number of concepts: 150 0.321
Number of concepts: 200 0.34
Number of concepts: 250 0.358
Number of concepts: 300 0.362
Number of concepts: 350 0.363
Number of concepts: 400 0.361


In [ ]:
best_index = lsi_model.retrieve_docs('lawyer office', normalization='stem')
df.iloc[best_index][['naics', 'title']]